<a href="https://colab.research.google.com/github/rhoanlee/MLP/blob/main/MLP%EB%A1%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 케라스의 texts_to_matrix() 이해하기

In [ ]:
texts = ['먹고 싶은 사과', '먹고 싶은 바나나', '길고 노란 바나나 바나나', '저는 과일이 좋아요']

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
print(tokenizer.word_index)

{'바나나': 1, '먹고': 2, '싶은': 3, '사과': 4, '길고': 5, '노란': 6, '저는': 7, '과일이': 8, '좋아요': 9}


In [ ]:
print(tokenizer.texts_to_matrix(texts, mode = 'count'))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


In [ ]:
print(tokenizer.texts_to_matrix(texts, mode = 'binary'))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


In [ ]:
print(tokenizer.texts_to_matrix(texts, mode = 'tfidf').round(2))

[[0.   0.   0.85 0.85 1.1  0.   0.   0.   0.   0.  ]
 [0.   0.85 0.85 0.85 0.   0.   0.   0.   0.   0.  ]
 [0.   1.43 0.   0.   0.   1.1  1.1  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.1  1.1  1.1 ]]


In [ ]:
print(tokenizer.texts_to_matrix(texts, mode = 'freq').round(2))

[[0.   0.   0.33 0.33 0.33 0.   0.   0.   0.   0.  ]
 [0.   0.33 0.33 0.33 0.   0.   0.   0.   0.   0.  ]
 [0.   0.5  0.   0.   0.   0.25 0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.33 0.33 0.33]]


# 2. 20개 뉴스 그룹(Twenty Newsgroups) 데이터에 대한 이해

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 48.1 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 448 kB 45.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import gspread
from google.colab import drive
drive.mount('/content/drive')

train = '/content/drive/My Drive/mbti/mbti_train.xlsx'
test= '/content/drive/My Drive/mbti/mbti_test.xlsx'

import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_data = pd.read_excel(train)
test_data = pd.read_excel(test)

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [ ]:
train_data['내용'] = train_data['내용'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['내용'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거
print('전처리 후 훈련용 샘플의 개수 :',len(test_data))

test_data['내용'] = test_data['내용'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['내용'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다',
             '인프피','인픕','인픞','인프제','인티피','인팁','인팊','인티제','잇프피','잇픕','잇픞','잇프제',
             '잇티피','잇팁','잇팊','잇티제','엔프피','엔픕','엔픞','엔프제','엔티피','엔팁','엔팊','엔티제',
             '엣프피','엣픕','엣픞','엣프제','엣티피','엣팁','엣팊','엣티제']

okt = Okt()

train_text = []
for sentence in tqdm(train_data['내용']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    train_text.append(stopwords_removed_sentence)

test_text = []
for sentence in tqdm(test_data['내용']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    test_text.append(stopwords_removed_sentence)


전처리 후 훈련용 샘플의 개수 : 2400
전처리 후 테스트용 샘플의 개수 : 1990


100%|██████████| 1990/1990 [00:13<00:00, 151.28it/s]


In [ ]:

train_label = train_data['label']
test_label = test_data['label']

In [ ]:
max_words = 10000
num_classes = 20

In [ ]:
def prepare_data(train_data, test_data, mode): # 전처리 함수
    tokenizer = Tokenizer(num_words = max_words) # max_words 개수만큼의 단어만 사용한다.
    tokenizer.fit_on_texts(train_data)
    X_train = tokenizer.texts_to_matrix(train_data, mode=mode) # 샘플 수 × max_words 크기의 행렬 생성
    X_test = tokenizer.texts_to_matrix(test_data, mode=mode) # 샘플 수 × max_words 크기의 행렬 생성
    return X_train, X_test, tokenizer.index_word

In [ ]:
X_train, X_test, index_to_word = prepare_data(train_text, test_text, 'binary') # binary 모드로 변환
y_train = to_categorical(train_label, num_classes) # 원-핫 인코딩
y_test = to_categorical(test_label, num_classes) # 원-핫 인코딩

In [ ]:
print('훈련 샘플 본문의 크기 : {}'.format(X_train.shape))
print('훈련 샘플 레이블의 크기 : {}'.format(y_train.shape))
print('테스트 샘플 본문의 크기 : {}'.format(X_test.shape))
print('테스트 샘플 레이블의 크기 : {}'.format(y_test.shape))

훈련 샘플 본문의 크기 : (6878, 10000)
훈련 샘플 레이블의 크기 : (6878, 20)
테스트 샘플 본문의 크기 : (1990, 10000)
테스트 샘플 레이블의 크기 : (1990, 20)


In [ ]:
print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))
print('빈도수 상위 9999번 단어 : {}'.format(index_to_word[9999]))

빈도수 상위 1번 단어 : .
빈도수 상위 9999번 단어 : 이어폰


# 3. 다층 퍼셉트론(Multilayer Perceptron, MLP)을 사용하여 텍스트 분류하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
def fit_and_evaluate(X_train, y_train, X_test, y_test):
    model = Sequential()
    model.add(Dense(256, input_shape=(max_words,), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.1)
    score = model.evaluate(X_test, y_test, batch_size=128, verbose=0)
    return score[1]

In [ ]:
modes = ['binary', 'count', 'tfidf', 'freq'] # 4개의 모드를 리스트에 저장.

for mode in modes: # 4개의 모드에 대해서 각각 아래의 작업을 반복한다.
    X_train, X_test, _ = prepare_data(train_text, test_text, mode) # 모드에 따라서 데이터를 전처리
    score = fit_and_evaluate(X_train, y_train, X_test, y_test) # 모델을 훈련하고 평가.
    print(mode+' 모드의 테스트 정확도:', score)

Epoch 1/5
49/49 [==============================] - 3s 22ms/step - loss: 1.5281 - accuracy: 0.4861 - val_loss: 0.6994 - val_accuracy: 0.6119
Epoch 2/5
49/49 [==============================] - 0s 10ms/step - loss: 0.7305 - accuracy: 0.5901 - val_loss: 0.6776 - val_accuracy: 0.6003
Epoch 3/5
49/49 [==============================] - 0s 10ms/step - loss: 0.5705 - accuracy: 0.7076 - val_loss: 0.7083 - val_accuracy: 0.5799
Epoch 4/5
49/49 [==============================] - 0s 10ms/step - loss: 0.4483 - accuracy: 0.7948 - val_loss: 0.8036 - val_accuracy: 0.5785
Epoch 5/5
49/49 [==============================] - 0s 10ms/step - loss: 0.3308 - accuracy: 0.8604 - val_loss: 0.9743 - val_accuracy: 0.5538
binary 모드의 테스트 정확도: 0.568844199180603
Epoch 1/5
49/49 [==============================] - 1s 14ms/step - loss: 1.5334 - accuracy: 0.4871 - val_loss: 0.7008 - val_accuracy: 0.6308
Epoch 2/5
49/49 [==============================] - 1s 11ms/step - loss: 0.7506 - accuracy: 0.5851 - val_loss: 0.6875 - val